In [1]:
!pip install pymupdf pillow pdfplumber pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 22.8 MB/s eta 0:00:00


In [3]:
from google.colab import files

# Upload the file
uploaded = files.upload()


Saving amc-annual-report---2015---16.pdf to amc-annual-report---2015---16.pdf


In [4]:
import fitz
import pdfplumber
import pandas as pd
import io
from PIL import Image
import os


def extract_tables(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            for table in page.extract_tables():
                if table:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    tables.append(df)
    return tables



def extract_images_from_page(document, page, page_num):
    image_list = page.get_images(full=True)
    images = []
    for img_index, img in enumerate(image_list):
        xref = img[0]
        base_image = document.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        image = Image.open(io.BytesIO(image_bytes))

        # Save the image
        image_path = f"image_page{page_num + 1}_{img_index}.{image_ext}"
        with open(image_path, "wb") as img_file:
            image.save(img_file)
        images.append(image_path)
        print(f"Image {img_index + 1} on Page {page_num + 1} saved as {image_path}.")
    return images



def process_pdf(pdf_path):
    if not os.path.exists(pdf_path):
        print(f"The file {pdf_path} does not exist.")
        return

    # Open the PDF file
    document = fitz.open(pdf_path)

    all_tables = extract_tables(pdf_path)
    all_images = []

    # Create final_images directory if it doesn't exist
    final_images_dir = "final_images"
    if not os.path.exists(final_images_dir):
        os.makedirs(final_images_dir)

    # Iterate through each page
    for page_num in range(len(document)):
        page = document.load_page(page_num)

        # Extract images
        images = extract_images_from_page(document, page, page_num)
        all_images.extend(images)

    # Save all tables into one CSV file
    final_csv = pd.concat(all_tables,axis=1)
    final_csv.to_csv("final_csv.csv", index=False)
    print("All tables saved into final_csv.csv")

    print("\nImages:")
    for image in all_images:
        # Move images to final_images directory
        image_name = os.path.basename(image)
        final_image_path = os.path.join(final_images_dir, image_name)
        os.rename(image, final_image_path)
        print(f"{image_name} moved to final_images directory.")

if __name__ == "__main__":
    pdf_path = list(uploaded.keys())[0]
    process_pdf(pdf_path)





Image 1 on Page 1 saved as image_page1_0.jpeg.
Image 1 on Page 2 saved as image_page2_0.jpeg.
Image 2 on Page 2 saved as image_page2_1.jpeg.
Image 3 on Page 2 saved as image_page2_2.jpeg.
Image 4 on Page 2 saved as image_page2_3.jpeg.
Image 5 on Page 2 saved as image_page2_4.jpeg.
Image 6 on Page 2 saved as image_page2_5.jpeg.
Image 7 on Page 2 saved as image_page2_6.jpeg.
Image 8 on Page 2 saved as image_page2_7.jpeg.
Image 9 on Page 2 saved as image_page2_8.jpeg.
Image 10 on Page 2 saved as image_page2_9.jpeg.
Image 11 on Page 2 saved as image_page2_10.jpeg.
Image 1 on Page 122 saved as image_page122_0.jpeg.
Image 2 on Page 122 saved as image_page122_1.png.
Image 3 on Page 122 saved as image_page122_2.png.
Image 4 on Page 122 saved as image_page122_3.png.
Image 5 on Page 122 saved as image_page122_4.png.
Image 6 on Page 122 saved as image_page122_5.png.
Image 7 on Page 122 saved as image_page122_6.png.
Image 8 on Page 122 saved as image_page122_7.png.
Image 9 on Page 122 saved as ima

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
